In [1]:
import numpy as np
import pandas as pd
from konlpy.tag import Mecab
import hanja
import re
import string
import operator
import random
import matplotlib.pyplot as plt
import itertools
import cnouns as cn
import check_utils as cu
from sklearn.metrics import adjusted_rand_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_samples, silhouette_score
from time import time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from datetime import datetime
from sklearn.decomposition import PCA

# Load Data

In [2]:
# test = 1
test = 2

In [3]:
if(test==1):
    topics = {
          0: u'올림픽',
          1: u'테러', 
          2: u'브렉시트', 
          3: u'미국 금리',
          4: u'바이러스', 
          5: u'미국대선,힐러리,트럼프', 
          6: u'시리아 전쟁, 난민'
         }
    train_df = pd.read_pickle("../datastore/international.p")
    n_clusters = len(topics)
elif(test==2):    
    train_df = pd.read_pickle("../datastore/weekly_2.p")
    n_clusters = 30

# Preprocessing

In [4]:
train_df['target_str'] = [cn.tokenize(row.title + " " + row.content) for idx, row in train_df.iterrows()]

In [5]:
vectorizer = TfidfVectorizer(lowercase=False, 
                             max_features=None,
                             max_df=1.0, 
                             min_df=1,
                             use_idf=True,
                             smooth_idf=True,
                             sublinear_tf=False)

In [6]:
x_list = vectorizer.fit_transform(train_df.target_str)

# Clustering

In [7]:
t0 = time()
km = KMeans(n_clusters=n_clusters, n_jobs=-1)
km.fit(x_list)
clusters = km.labels_.tolist()
train_df['cluster'] = clusters
print("Done in %0.3fs." % (time() - t0))

KeyboardInterrupt: 

In [8]:
train_df.to_pickle("../datastore/clusters.p")

In [9]:
train_df[:10]

,_id,category,content,description,provider,providerNewsID,publishedAt,title,target_str
0,57e2c849149c2181df5b92aa,nativeAD > CJ,CJ그룹이 7일부터 하반기 신입사원 1700명을 공개 채용한다. 작년보다 18% 늘...,,chosun,2016090701420,2016-09-07 11:31:00,"CJ그룹, 하반기 신입사원 1700명 공개 채용",그룹NNG 하반기NNG 신입NNG 사원NNG 공개NNG 채용NNG 그룹NNG 하반기...
1,57e2c849149c2181df5b92ac,사회 > 카드뉴스,흙수저 출신 20대 억만장자의 '엉터리' 조언 기사보기 : http://goo.g...,,chosun,2016090701310,2016-09-07 10:59:00,[카드뉴스] 그의 수퍼카 부가티는 개미들의 눈물로 달렸을까요?,수퍼NNP 카NNP 부가티NNP 개미NNG 눈물NNG 흙NNG 수저NNG 출신NNG...
2,57e2c849149c2181df5b92ad,사회 > 카드뉴스,호주 역사상 처음!\n\n[ 조선일보 카드뉴스가 더 보고 싶다면 ?] \n\n[ 조...,,chosun,2016090701283,2016-09-07 10:49:00,[카드뉴스] 열한살 소년이 지폐를 바꿨습니다,소년NNG 지폐NNG 호주NNP 역사NNG 호주NNP역사NNG 소년NNG이JKS지폐...
3,57e2c849149c2181df5b92ae,사회 > 카드뉴스,한국프로야구 승부조작 \n[ 조선일보 카드뉴스가 더 보고 싶다면 ?] \n\n[ 조...,,chosun,2016090701173,2016-09-07 10:23:00,[카드뉴스] '원조 승부조작' 前 야구선수의 참회,원조NNG 승부조작NNP 야구NNG 선수NNG 참회NNG 한국NNP 프로야구NNP ...
4,57e2c849149c2181df5b92b0,사설ㆍ칼럼 > 내부칼럼 > 사설,추미애 더불어민주당 대표가 5일 취임 이후 첫 국회 교섭단체 연설에서 청와대에 '비...,,chosun,2016090603334,2016-09-07 03:23:00,[사설] 경제 앞길 막은 야당이 경제 失政 비판할 자격 있나,경제NNG 앞길NNG 야당NNG 경제NNG 실정NNG 비판NNG 자격NNG 추미NN...
5,57e2c849149c2181df5b92b1,사설ㆍ칼럼 > 내부칼럼 > 사설,금융 공기업에 파견 근무 중인 김모 부장검사가 자신의 스폰서 역할을 해온 고교 동창...,,chosun,2016090603336,2016-09-07 03:19:00,"[사설] 檢察선 또 스폰서 검사, 法院선 또 돈 받고 재판",검찰NNG 스폰서NNG 검사NNG 법원NNG 돈NNG 재판NNG 금융NNG 공기업N...
6,57e2c84a149c2181df5b92b2,사설ㆍ칼럼 > 내부칼럼 > 사설,의료기관에서 C형 간염 집단감염 사태가 잇따르자 정부가 6일 C형 간염을 전수(全數...,,chosun,2016090603338,2016-09-07 03:18:00,"[사설] C형 간염, 의료기 再사용 병·의원 명단 공개해야",간염NNG 의료기NNG 사용NNG 병NNG 의원NNG 명단NNG 공개NNG 의료NN...
7,57e2c84a149c2181df5b92b4,사설ㆍ칼럼 > 외부칼럼 > 시론ㆍ기고 > 朝鮮칼럼 The Column,대한민국을 건국한 동력은 무엇인가? 요즈음 벌어지고 있는 국가 정체성 논란의 핵심에...,최근 벌어진 국가 정체성 논란엔 '건국 동력 무엇인가' 핵심 있어\n민주주의·자본주...,chosun,2016090603347,2016-09-07 03:17:00,[朝鮮칼럼 The Column] 대한민국 건국 = 항일 + 반공,대한민국NNP 건국NNG 항일NNG 반공NNG 대한민국NNP 건국NNG 동력NNG ...
8,57e2c84a149c2181df5b92b5,사설ㆍ칼럼 > 내부칼럼 > 만물상,부산 초읍동 어린이대공원 뒤 백양산을 오르다 보면 높이 27m 거대한 석벽을 마주한...,,chosun,2016090603367,2016-09-07 03:16:00,[만물상] 음수사원(飮水思源),수사원NNG 글자NNG 눈NNG 물NNG 때NNG 수원지NNG 사람NNG 노고NNG...
9,57e2c84a149c2181df5b92b6,사설ㆍ칼럼 > 외부칼럼 > 정민의 세설신어,유관현(柳觀鉉·1692~1764)은 1759년 필선(弼善)의 직책으로 사도세자를 3...,,chosun,2016090603374,2016-09-07 03:15:00,[정민의 世說新語] [382] 난자이사 (難者二事),난자NNG 이사NNG 대목NNG 가난NNG 부자NNG 의리NNG 돈NNG 공NNG ...


# Scoring

In [10]:
adjusted_rand_score(train_df.topic, km.labels_)

AttributeError: 'DataFrame' object has no attribute 'topic'

In [ ]:
cu.show_cluster(train_df, range(n_clusters))

In [ ]:
cu.match_cluster_topic(train_df, False, topics, n_clusters)

In [ ]:
cu.match_cluster_topic(train_df, True, topics, n_clusters)

In [ ]:
cu.show_cartesian(train_df, n_clusters)